<a href="https://colab.research.google.com/github/TheMortalSea/TheMortalSea.github.io/blob/main/w04_postgis2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spatial Joins Exercises

Here\'s a reminder of some of the functions we have seen. Hint: they
should be useful for the exercises!

-   `sum(expression)`: aggregate to
    return a sum for a set of records
-   `count(expression)`: aggregate to
    return the size of a set of records
-   `ST_Area(geometry)` returns the
    area of the polygons
-   `ST_AsText(geometry)` returns WKT `text`
-   `ST_Contains(geometry A, geometry B)` returns the true if geometry A contains geometry B
-   `ST_Distance(geometry A, geometry B)` returns the minimum distance between geometry A and
    geometry B
-   `ST_DWithin(geometry A, geometry B, radius)` returns the true if geometry A is radius distance or less from geometry B
-   `ST_GeomFromText(text)` returns `geometry`
-   `ST_Intersects(geometry A, geometry B)` returns the true if geometry A intersects geometry B
-   `ST_Length(linestring)` returns the length of the linestring
-   `ST_Touches(geometry A, geometry B)` returns the true if the boundary of geometry A touches geometry B
-   `ST_Within(geometry A, geometry B)` returns the true if geometry A is within geometry B


Uncomment and run the following cell to install the required packages.


In [ ]:
# %pip install duckdb leafmap lonboard
import duckdb
import leafmap

Download the [nyc_data.zip](https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip) dataset using leafmap. The zip file contains the following datasets. Create a new DuckDB database and import the datasets into the database. Each dataset should be imported into a separate table.

- nyc_census_blocks
- nyc_homicides
- nyc_neighborhoods
- nyc_streets
- nyc_subway_stations

In [ ]:

url = "https://open.gishub.org/data/duckdb/nyc_data.db.zip"
leafmap.download_file(url, unzip=True)

Downloading...
From: https://open.gishub.org/data/duckdb/nyc_data.db.zip
To: /content/nyc_data.db.zip
100%|██████████| 8.60M/8.60M [00:00<00:00, 22.5MB/s]


Extracting files...


'/content/nyc_data.db.zip'

In [ ]:
con = duckdb.connect('nyc_data.db')

In [ ]:
con.install_extension('spatial')
con.load_extension('spatial')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
con.sql("SHOW TABLES;")

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ nyc_census_blocks   │
│ nyc_homicides       │
│ nyc_neighborhoods   │
│ nyc_streets         │
│ nyc_subway_stations │
└─────────────────────┘

1. **What subway station is in \'Little Italy\'? What subway route is it on?**

In [ ]:
con.sql("""
SELECT s.name, s.routes
FROM nyc_subway_stations AS s
JOIN nyc_neighborhoods as n ON ST_Contains(
  n.geom,
  s.geom
)
WHERE n.name = 'Little Italy'
""")

┌───────────┬─────────┐
│   NAME    │ ROUTES  │
│  varchar  │ varchar │
├───────────┼─────────┤
│ Spring St │ 6       │
└───────────┴─────────┘

2. **What are all the neighborhoods served by the 6-train?** (Hint: The `routes` column in the `nyc_subway_stations` table has values like \'B,D,6,V\' and \'C,6\')


In [ ]:
con.sql("""
SELECT DISTINCT n.name
FROM nyc_subway_stations AS s
JOIN nyc_neighborhoods AS n ON ST_Contains(
  n.geom,
  s.geom
)
WHERE strpos(s.routes, '6') > 0
""")

┌────────────────────┐
│        NAME        │
│      varchar       │
├────────────────────┤
│ Chinatown          │
│ Greenwich Village  │
│ Murray Hill        │
│ Midtown            │
│ Upper East Side    │
│ Yorkville          │
│ Hunts Point        │
│ South Bronx        │
│ Soundview          │
│ Parkchester        │
│ Financial District │
│ Little Italy       │
│ Gramercy           │
│ East Harlem        │
│ Mott Haven         │
├────────────────────┤
│      15 rows       │
└────────────────────┘

3. **After 9/11, the \'Battery Park\' neighborhood was off limits for several days. How many people had to be evacuated?**

In [ ]:
con.sql("""
SELECT SUM(n.popn_total)
FROM nyc_census_blocks AS n
JOIN nyc_neighborhoods AS s
ON ST_Contains(s.geom, ST_Centroid(n.geom))
WHERE s.name = 'Battery Park'
""")

┌───────────────────┐
│ sum(n.popn_total) │
│      int128       │
├───────────────────┤
│             10460 │
└───────────────────┘

4. **What neighborhood has the highest population density (persons/km2)?**


In [ ]:
con.sql("""
SELECT SUM(n.popn_total) / (ST_Area(s.geom) / 1000000.0) AS density, s.name
FROM nyc_census_blocks AS n
JOIN nyc_neighborhoods AS s
ON ST_Contains(s.geom, ST_Centroid(n.geom))
GROUP BY s.name, S.geom
ORDER BY density DESC
LIMIT 1
""")

┌───────────────────┬───────────────────┐
│      density      │       NAME        │
│      double       │      varchar      │
├───────────────────┼───────────────────┤
│ 52048.48348771367 │ North Sutton Area │
└───────────────────┴───────────────────┘

When you're finished, you can check your answers [here](https://postgis.net/workshops/postgis-intro/joins_exercises.html).

# Ship-to-Ship Transfer Detection

Now for a less structured exercise. We're going to look at ship-to-ship transfers. The idea is that two ships meet up in the middle of the ocean, and one ship transfers cargo to the other. This is a common way to avoid sanctions, and is often used to transfer oil from sanctioned countries to other countries. We're going to look at a few different ways to detect these transfers using AIS data.

In [ ]:
%pip install duckdb duckdb-engine jupysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.9 MB/s eta 0:00:00


In [ ]:
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

In [ ]:
%%sql
INSTALL httpfs;
LOAD httpfs;
INSTALL spatial;
LOAD spatial;

,Success


## Step 1

Create a spatial database using the following AIS data:

https://storage.googleapis.com/qm2/casa0025_ships.csv

Each row in this dataset is an AIS 'ping' indicating the position of a ship at a particular date/time, alongside vessel-level characteristics.

It contains the following columns:
* `vesselid`: A unique numerical identifier for each ship, like a license plate
* `vessel_name`: The ship's name
* `vsl_descr`: The ship's type
* `dwt`: The ship's Deadweight Tonnage (how many tons it can carry)
* `v_length`: The ship's length in meters
* `draught`: How many meters deep the ship is draughting (how low it sits in the water). Effectively indicates how much cargo the ship is carrying
* `sog`: Speed over Ground (in knots)
* `date`: A timestamp for the AIS signal
* `lat`: The latitude of the AIS signal (EPSG:4326)
* `lon`: The longitude of the AIS signal (EPSG:4326)

Create a table called 'ais' where each row is a different AIS ping, with no superfluous information. Construct a geometry column.

Create a second table called 'vinfo' which contains vessel-level information with no superfluous information.

You can set a spatial index on each of these tables as follows:

`CREATE INDEX index_name ON table_name USING RTREE(geom);`

In [ ]:
%sql ROLLBACK;

,Success


In [ ]:
%%sql DROP TABLE ais;

,Success


In [ ]:
%%sql DROP TABLE vinfo;

,Success


In [ ]:
%%sql
CREATE TABLE ais AS
SELECT
    vesselid,
    date,
    sog,
    lat,
    lon,
    ST_Point(lon, lat) AS GEOMETRY
FROM read_csv_auto('https://storage.googleapis.com/qm2/casa0025_ships.csv');

,Success


In [ ]:
%%sql
CREATE TABLE vinfo AS
SELECT DISTINCT vesselid, vessel_name, vsl_descr, dwt, v_length
FROM read_csv_auto('https://storage.googleapis.com/qm2/casa0025_ships.csv');

,Success


In [ ]:
%%sql
SELECT (*)
FROM vinfo
LIMIT 5

,vesselid,vessel_name,vsl_descr,dwt,v_length
0,301537,Omskiy 86,general cargo with container capacity,3201.0,108.0
1,276630,Omskiy- 127,general cargo with container capacity,3177.0,108.0
2,256167,Omskiy- 128,general cargo,3174.0,108.0
3,265523,Omskiy- 206,general cargo with container capacity,2835.0,114.0
4,246401,Omskiy-106,general cargo,3191.0,108.0


In [ ]:
%%sql
CREATE INDEX ais_geom_idx ON ais USING RTREE(GEOMETRY);


,Success


In [ ]:
%%sql
CREATE INDEX vinfo_vesselid_idx ON vinfo(vesselid);

,Success


## Step 2

Use a spatial join to identify ship-to-ship transfers in this dataset.
Two ships are considered to be conducting a ship to ship transfer IF:

* They are within 500 meters of each other
* For more than two hours
* And their speed is lower than 1 knot

Some things to consider: make sure you're not joining ships with themselves. Try working with subsets of the data first while you try different things out.

In [ ]:
%%sql

SELECT a1.vesselid AS ship1, a2.vesselid AS ship2, a1.date AS start, a2.date AS end
FROM ais AS a1
JOIN ais AS a2
ON ST_DWithin(a1.GEOMETRY, a2.GEOMETRY, 500) AND ABS(EXTRACT(EPOCH FROM (a2.date - a1.date))) > 7200 AND a1.vesselid <> a2.vesselid AND a1.sog > 1 AND a2.sog > 1
LIMIT 20

,ship1,ship2,start,end
0,323648,350053,2022-06-28 14:31:37,2022-07-25 02:53:29
1,323648,350053,2022-06-28 14:41:56,2022-07-25 02:53:29
2,323648,350053,2022-06-28 14:51:59,2022-07-25 02:53:29
3,323648,350053,2022-06-28 15:02:17,2022-07-25 02:53:29
4,323648,350053,2022-07-01 21:45:32,2022-07-25 02:53:29
5,142540,350053,2022-08-18 13:14:19,2022-07-25 02:53:29
6,142540,350053,2022-08-18 13:30:57,2022-07-25 02:53:29
7,319402,350053,2022-08-14 16:24:50,2022-07-25 02:53:29
8,319402,350053,2022-08-14 16:53:53,2022-07-25 02:53:29
9,319402,350053,2022-08-14 17:18:08,2022-07-25 02:53:29
